company-name 公司名稱
job-opening 職缺
requirement 應備技能

In [23]:
import requests
from bs4 import BeautifulSoup
import os
import json
import pandas as pd

#os.mkdir("./104_job")

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
    "Referer": "https://www.104.com.tw/job"
}


url = "https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90%E5%B8%AB&jobcatExpansionType=0&order=14&asc=0&page={}&mode=s&jobsource=2018indexpoc"
page = 1

for i in range(0,3):
    res = requests.get(url.format(page),headers=headers)

    html = res.text

    soup = BeautifulSoup(html, "html.parser")

    job = soup.select("article[class='b-block--top-bord job-list-item b-clearfix js-job-item']")
    job_link = soup.select("a[class='js-job-link']")
    
    
    for j , k in zip(job, job_link):
        company_name = j["data-cust-name"]
        job_name = j["data-job-name"]
        job_url = "https:" + k["href"]
        
        job_js_url = "https://www.104.com.tw/job/ajax/content/{}"

        job_res = requests.get(url=job_js_url.format(k["href"][21:26]), headers=headers)

        job = json.loads(job_res.text)
        
        area = job["data"]["jobDetail"]["addressRegion"]
        
        job_Description = job["data"]["jobDetail"]["jobDescription"]
        job_Description = job_Description.replace(",","，")

        job_skill = ""
        tmp_skill = job["data"]["condition"]["specialty"]

        for i in range(len(tmp_skill)):
            job_skill = job_skill + " " + tmp_skill[i]["description"]
        
        others = job["data"]['condition']['other']
        
        
        try:
            with open("./104_job/{}.txt".format(company_name),"w",encoding="utf-8") as f:
                f.write(company_name+"\n====="+job_name+"\n====="+area+"\n====="+job_Description+"\n====="+job_skill+"\n====="+others+"\n====="+job_url)
                
        except OSError:
            pass
        print(company_name)
        print(area)
        print(job_name)
        print(job_url)
        print(job_Description)
        print(job_skill)
        print(others)
        print("=====================================================")

    page += 1

    
file_list = os.listdir("./104_job")
df = pd.DataFrame(columns = ["company_name","job_name","area","job_Description","skills","others","104_url"])

job_row_data = []
for each_job in file_list:
    with open("./104_job"+"/"+each_job, "r", encoding="utf-8") as f:
        tmp_job_content = f.read()
    job_content = tmp_job_content.replace("\n","")
    tmp_job_row_data = job_content.split("=====")
    job_row_data.append(tmp_job_row_data)
#print(job_row_data)

df = df.append(pd.DataFrame(job_row_data,columns = ["company_name","job_name","area","job_Description","skills","others","104_url"]))
df.to_csv("./job.csv",index=0, encoding="utf-8")


唯渥國際有限公司
菲律賓
產品運營資料分析師
https://www.104.com.tw/job/6yf8o?jobsource=2018indexpoc
產品運營資料分析師
資料分析師職責：
1.	管理資料，包括創建，更新和刪除；
2.	導入資料驅動決策相關分析，必要時需要跨部門合作；
3.	調試和停用資料集；
4.	根據準則處理機密資料和資訊；
5.	幫助開發報告和分析；
6.	管理和設計報告環境，包括資料來源，安全性和中繼資料；
7.	配合資料倉庫識別和修訂報告要求；
8.	配合資料完整性和標準化的計畫；
9.	評估測試並實施新的或升級的軟體；
10.	從單個或多個系統生成報告；
11.	對報告資料庫環境和報告進行故障排除；
12.	評估資料分析系統的更改和更新；
13.	在新報告和儀錶板上對內部員工進行教育訓練；
14.	提供有關資料存儲結構，資料採擷和資料清理的技術專長；
15.	提供各單位業務相關資料撈取工具與報表；

 Python MS SQL Tableau
資料分析師要求：
1.	擁有資工、統計、資料等領域大學或以上學位；
2.	作為資料分析師或相關領域的工作經驗2年以上工作經驗，或相關領域碩士學歷；
3.	與產品經理共事以評估潛在風險的能力；
4.	能夠利用現有工具和資料庫進行分析，並提供應用軟體相關解決方案建議；
5.	能夠將業務需求轉換為非技術性的非專業術語；
6.	精熟於管理大型資料庫的方法與流程；
7.	在處理大型資料集和關聊式資料庫方面具有豐富的經驗；
8.	瞭解中繼資料標準；
9.	高水準的書面和口頭交流能力；
10.	至少需要擅長一種資料分析語言，ex，R,SQL,.Python等；
11.	至少需要精熟一種圓形化分析工具，ex，Tableau，Excel，GA
12.	對於AL、ML等開發運用有經驗者，加分；
13.	有成功的GA商業應用經驗者，加分；
14.	有博弈產業經驗者，加分；
15.	有BigQuery等雲端大資料分析經驗者，加分
薪資：面議

烏龜移動科技股份有限公司
台北市松山區
資料分析師
https://www.104.com.tw/job/63jf9?jobsource=2018indexpoc
蝙蝠移動的核心服務是提供客戶專屬的私有化 DMP（Data Management Platform 數據分析管理

亞洲指標數位行銷顧問股份有限公司
台北市松山區
資料分析師 (Data Analyst)
https://www.104.com.tw/job/6k2x6?jobsource=2018indexpoc
i-Buzz的核心理念是數據推動行銷策略，透過網路語料觀察與混搭資料研究進行開發，提供客戶深化的資料分析結果。另外欲透過自營的數位產品，挖掘使用者的行為數據，提供產品更好的體驗。

職務內容如下：
1. 文字探勘
2. 進行資料建模
3. 資料視覺化處理
4. 開發預測模型
5. 發展推薦演算法、個人化演算法

此職缺需熟悉統計分析、Python或R語言、Machine Learning、MongoDB、SQL。誠摯歡迎符合以上條件，且具責任心、勇於接受挑戰者的加入。
 Python R
1. 具一年以上資料分析經驗。
2. 具邏輯力、溝通能力，可獨立進行會議簡報。
3. 個性主動積極，主動學習者。
4. 能獨立處理及解決問題。
5.知道如何使用 git 尤佳。

逢甲大學_地理資訊系統研究中心
台中市西屯區
資深資料分析師_統合決策處
https://www.104.com.tw/job/6ubxj?jobsource=2018indexpoc
1. 使用統計，數學，機器學習或deep learning等方法分析專案計畫產生的資料。
2. 將問題與需求轉化為可分析的條件以及狀態。
3. 定義專案分析的範圍以及方法論，定義假說以及指標，調整設計數據模型。
4. 定義解決方法步驟的優先度。
5. 設計建立分析所需工具。
6. 專案成果彙整。
 Python R Excel Outlook PowerPoint Word
1.精通統計理論及機器學習相關知識。
2.熟悉Python或R之數據分析機器學習相關套件。
3.熟悉SQL資料庫語法及工具。
4.良好的溝通能力，能解釋複雜分析方法給予其他領域的專家了解。
5.熟悉電腦視覺演算法(object detection、segmentation、classification)及數位影像處理。
6.熟悉OpenCV、tensorflow或pytorch等常用套件- 具以下相關經驗者佳:
(1) 資料分析專案執行規劃
(2) 參與Kaggle或datathon
(3) Computer vision、Text mining等實作經

艾奕康工程顧問股份有限公司
台北市信義區
資料庫與數據分析師 Data and Analytics Engineer(建築與人居環境部-台北)
https://www.104.com.tw/job/6y5r2?jobsource=jolist_b_relevance
AECOM數位解決方案團隊正在尋找熟悉建築與都市工程產業的資料庫與數據工程師，應徵者將與AECOM亞太區各地的專案和市場行銷團隊合作，協助團隊研究並建構資料庫結構與數據分析機制，打造支援雲端協作、行為分析以及資料視覺化的AR／VR／Web設計輔助工具，一同為AEC（Architecture， Engineering and Construction）產業帶來革新的工作流程與設計觀點。 

應徵者需具備專案管理與產品規畫的實務經驗，並且可獨立完成資料庫建構、資料擷取、導入、分析解讀和應用等工作，實務上需熟悉資料結構、分析指標、資料庫語言SQL、雲端儲存(Azure， AWS)、前端和後端開發語言（如：HTML5、ASP.NET、Python等）。若同時具備資料視覺化或機器學習等技術概念者尤佳。 

AECOM為構築更美好的世界而生，我們為全球150多個國家的政府機構與企業組織提供規劃設計、財務策劃與基礎設施營運的全方位服務。身為一個高度專業整合的企業，AECOM具備完整的全球專業知識網路與國際經驗，可以協助客戶解決高度複雜的業務挑戰。從地標建築與基礎設施、永續發展與環境規劃以至於國防建設與國家發展，AECOM提供多樣化的服務，身為美國Fortune 500大企業與Engineering News-Record排名全球第一的設計顧問公司。



Basic
1.	可獨立完成資料庫建置、資料擷取、導入、分析解讀和應用等工作 
2.	熟悉資料結構、分析指標、資料庫語言SQL和雲端儲存(Azure, AWS) 
3.	熟悉前後端開發語言（如：HTML5、ASP.NET、Python等）。 
4.	具備獨立操作的技術能力，同時擁有良好的合作精神和溝通技巧，且樂於互助和勇於承擔責任 
5.	喜愛具挑戰性與創新的工作環境 
6.	具備良好的中、英語溝通能力，樂於參與跨國／跨領域專案協作 
Plus
7.	具備相關專案管理與產品規畫的實務經驗 
8.	能擔任專案聯繫窗口，具備理解市場需求的能力，並能協助團隊推廣和行銷專案

欣興電子股份有限公司
新竹縣新豐鄉
【工業4.0】智慧製造程式設計工程師++熟悉VB.Net、C#++(新豐廠)
https://www.104.com.tw/job/6z913?jobsource=jolist_b_relevance
1. 負責製造相關系統-應用分析與程式開發。
2. 提升關鍵製程/設備/智動化運用-系統開發/維護。
3. 擴展大數據與AI人工智慧運用-系統開發/維護。
4. 專案之系統分析與程式設計業務。
★VB.Net、C#、MS SQL Server、系統分析與程式設計能力。
☆有MES系統維護經驗或現場製程經驗尤佳。
★Mongo DB、Python 大數據應用。
 Visual Basic .net MS SQL
1. 大學以上相關科系畢 
2. 溝通協調能力佳, 積極負責 
3. 表達能力良好, 抗壓性高
網銀國際股份有限公司
台中市西屯區
【研發中心】資料技術工程師
https://www.104.com.tw/job/6q8dn?jobsource=jolist_b_relevance
【工作內容】
● 各類 DB System 安裝及參數配置
● HBase ， Redis ， MySQL 維運
● 建立高可用性資料模式
● 效能調校、監控及異常處理
● GCP 雲端應用

【技能需求】
● Linux Base

 
優橋貿易股份有限公司
台北市中正區
系統開發軟體工程師（Java PHP HTML）
https://www.104.com.tw/job/6hfxn?jobsource=jolist_b_relevance
1. 與外包廠商一起建置、開發公司內部管理系統
2. 與外包廠商一起建置、規劃資料庫
3. 公司內部系統的設定・維護・優化及管理
4. 提高公司內部作業流程的效率，導入/評估新技術&產品
**請具體說明實際使用需求軟體的東西， 並請提供作品集參考， 謝謝!
 C C++ Java PHP HTML JavaScript
1. 高度積極，並且願意主動學習
2. 有良好的溝通能力和願意幫公司解決內部系統問題的熱忱 
3. 有ERP系統程式開發一年以上的實務經驗
4. 有公司內部系統相關知識及系統設計或開發之實務經驗尤佳 
5. 喜愛日本文化，對在日商工作有興趣尤佳  

【工作魅力】
1. 加班或假日出勤頻率低

ValueError: 7 columns passed, passed data had 14 columns